In [339]:
import pandas as pd
import numpy as np
from dstapi import DstApi

import ipywidgets as widgets
from ipywidgets import TwoByTwoLayout
from IPython.display import display,clear_output
import matplotlib.pyplot as plt
import time

import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [28]:
ind = DstApi('FOLK1A')

In [66]:
params = {'table':'FOLK1A',
         'format':"BULK",
         'variables':[{'code':'OMRÅDE','values':kom},
                     {'code':'KØN','values':['*']},
                     {'code':'ALDER','values':['*']},
                     {'code':'Tid','values':['*']},
                     {'code':'CIVILSTAND','values':['TOT']}]}
data= ind.get_data(params)

In [347]:
data = pd.read_csv('FOLK1A',index_col=[0])
data['TID'] = pd.to_datetime(data.reset_index()['TID'].apply(change_format))
#data.resample('Y').last()
data = data[data['KØN']=='I alt'][['TID','OMRÅDE','ALDER','INDHOLD']]
data['ÅR'] = data['TID'].dt.year
data = data.groupby(['ÅR','OMRÅDE','ALDER']).last()
data = data.reset_index()
data = data.drop(columns = ['TID'])
data['ÅR'] = data['ÅR'].astype(int)
data = data[data['ALDER'] != 'I alt']
data['ALDER'] = data['ALDER'].str.strip(' år')
data['ALDER'] = data['ALDER'].astype(int)

In [348]:
REG = pd.read_csv('REGK11.csv',index_col=0)
BEF = data

In [358]:
BEF.rename(columns={'ÅR':'TID'},inplace=True)
new_df = pd.merge(BEF, REG,  how='left', left_on=['OMRÅDE','TID'], right_on = ['OMRÅDE','TID'])
samlet_bef = new_df.groupby(['TID','OMRÅDE'])['INDHOLD'].sum()
new_df['GAMLE'] = new_df['INDHOLD']
new_df.loc[new_df['ALDER']<60,'GAMLE'] = 0
samlet_bef1 = new_df.groupby(['TID','OMRÅDE'])['GAMLE'].sum()
df_ = pd.merge(samlet_bef, samlet_bef1,  how='left', left_on=['OMRÅDE','TID'], right_on = ['OMRÅDE','TID'])
df_['ANDEL'] = df_['GAMLE']/df_['INDHOLD']
df_.rename(columns={'INDHOLD':'SAMLET BEFOLKNING'},inplace=True)
df_ = pd.merge(df_,REG,how='left', left_on=['OMRÅDE','TID'], right_on = ['OMRÅDE','TID'])

In [362]:
df_

,OMRÅDE,TID,SAMLET BEFOLKNING,GAMLE,ANDEL,SUNDHEDSUDGIFTER
0,Aabenraa,2008,60284,14561,0.241540,2635.0
1,Aalborg,2008,195145,42502,0.217797,2393.0
2,Aarhus,2008,298711,53213,0.178142,2394.0
3,Albertslund,2008,27566,5414,0.196401,2598.0
4,Allerød,2008,23515,5253,0.223389,2618.0
...,...,...,...,...,...,...
1595,Vejle,2023,120949,30525,0.252379,NaN
1596,Vesthimmerlands,2023,36431,11754,0.322637,NaN
1597,Viborg,2023,97731,27318,0.279522,NaN
1598,Vordingborg,2023,45441,16655,0.366519,NaN


In [380]:
def plot_graph1(change=None):
    plot_graph_1(output_widget)

def plot_graph2(change=None):
    plot_graph_2(output_widget2)

def plot_graph_1(output):
    year = year_slider.value
    municipal = municipal_slider.value
    
    # Filter the DataFrame based on the selected year and municipal
    filtered_df = data[data['TID'] == year]
    filtered_df = filtered_df[filtered_df['OMRÅDE'].isin(municipal)]

    # Create the bar plot
    plt.barh(filtered_df['ALDER'].unique(), filtered_df['INDHOLD'])
    plt.title(f"Values for year {year}")
    plt.xlabel("Antal")
    plt.ylabel("År")
    plt.tight_layout()

    # Display the graph in the output widget
    with output:
        output.clear_output(wait=True)
        display(plt.gcf())
        
    plt.clf()
    
def plot_graph_2(output):
    year = year_slider.value
    municipal = municipal_slider.value
    
    filtered_df = df_[df_['OMRÅDE'].isin(municipal)]
    
    xx = filtered_df['ANDEL']
    yy = filtered_df['SUNDHEDSUDGIFTER']
    
    plt.scatter(xx,yy)

    for i, txt in enumerate(filtered_df['TID'].unique()):
    #    plt.annotate(txt, (xx.values[i], yy.values[i]), textcoords='offset points')
        plt.annotate(txt, (xx.values[i], yy.values[i]))
        
    with output:
        output.clear_output(wait=True)
        display(plt.gcf())
        
    plt.clf()

# Create the year slider
year_slider = widgets.IntSlider(
    value=data['TID'].min(),
    min=data['TID'].min(),
    max=data['TID'].max(),
    step=1,
    description="Year:",
    continuous_update=True,
)

# Create the municipal slider
municipal_slider = widgets.SelectMultiple(
    options=data['OMRÅDE'].unique(),
    value=['Ærø'],
    description='Municipal:',
    disabled=False,
)

# Create the output widgets
output_widget = widgets.Output()
output_widget2 = widgets.Output()

# Observe the slider changes and update the graphs
year_slider.observe(plot_graph1, names='value')
municipal_slider.observe(plot_graph1, names='value')

year_slider.observe(plot_graph2, names='value')
municipal_slider.observe(plot_graph2, names='value')

# Create a 2x2 GridBox layout
layout = widgets.GridBox(
    children=[year_slider, municipal_slider, output_widget, output_widget2],
    layout=widgets.Layout(
        width='100%',
        grid_template_columns='50% 50%',
        grid_template_rows='25% 50%',
        grid_gap='10px',
        grid_template_areas='''
            "year_slider municipal_slider"
            "output_widget output_widget2"
        ''')
)

# Initialize the graphs
plot_graph1()
plot_graph2()

# Display the GridBox layout
display(layout)


GridBox(children=(IntSlider(value=2008, description='Year:', max=2023, min=2008), SelectMultiple(description='…

<Figure size 432x288 with 0 Axes>

In [1]:
from dstapi import DstApi

In [283]:
ind = DstApi('REGK11')

params = {'table':'REGK11',
         'format':"BULK",
         'variables':[{'code':'OMRÅDE','values':kom},
                     {'code':'PRISENHED','values':['INDL']},
                     {'code':'DRANST','values':['1']},
                     {'code':'FUNK1','values':['4']},
                     {'code':'ART','values':['TOT']},
                     {'code':'TID','values':['>2007']}]}
data_2 = ind.get_data(params)

data_2.drop(columns=['PRISENHED', 'DRANST', 'FUNK1', 'ART'],inplace=True)

data_2.rename(columns={'INDHOLD':'SUNDHEDSUDGIFTER'},inplace=True)

KeyError: "['PRISENHED', 'DRANST', 'FUNK1', 'ART'] not found in axis"